In [2]:
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
import requests
from bs4 import BeautifulSoup
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import deepxde as dde
import os
from pykrige.ok import OrdinaryKriging
import tensorflow as tf
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.stats import norm
import warnings
warnings.filterwarnings("ignore")


In [3]:
FIGURE_PATH = "C:/Users/vitas/Desktop/LE PINN/pinn.global.dust/pinn.global.dust/Code/figures/"
DATA_PATH = "C:/Users/vitas/Desktop/LE PINN/pinn.global.dust/pinn.global.dust/Data/"
DATA_LOAD_PATH = DATA_PATH + "original_data/"
INPUT_MODEL_PATH = DATA_PATH + "processed_data/"
MODEL_SAVE_PATH = DATA_PATH + "trained_models/"
RESULTS_PATH = DATA_PATH + "model_results/"
path_to_shapefile = "C:/Users/vitas/Desktop/LE PINN/pinn.global.dust/pinn.global.dust/ne_110m_admin_0_countries.shp"
world = gpd.read_file(path_to_shapefile)

In [4]:
with open("functions_training_model.py", 'r') as file:
    content = file.read()

# Execute the content of the .py file
exec(content)

In [5]:
df_simulated_Holocene = pd.read_csv(INPUT_MODEL_PATH + "df_simulation_Holocene.csv")

df_global_grid = pd.read_csv(INPUT_MODEL_PATH + "df_global_grid.csv")
df_wind = pd.read_csv(INPUT_MODEL_PATH + "df_wind.csv", usecols=['wind', 'latitude'])
latitude_wind, mean_wind = df_wind['latitude'].values / 90, df_wind['wind'].values / df_wind['wind'].max()


def wind_latitude(latitude):
    interpolated = wind_tf_interp(latitude, tf.convert_to_tensor(latitude_wind), tf.convert_to_tensor(mean_wind))
    return interpolated


tf_wind_latitude = tf.function(wind_latitude)


In [6]:

def training_points(df):
    data_observ_points = dde.data.DataSet(
        X_train=df[['lon', 'lat']].values / 90,
        y_train=df['log_dep_norm'].values.reshape(-1, 1),
        X_test=df[['lon', 'lat']].values / 90,
        y_test=df['log_dep_norm'].values.reshape(-1, 1),
        standardize=False)

    observe_u = dde.icbc.PointSetBC(data_observ_points.train_x,
                                    df['log_dep_norm'].values.reshape(-1, 1), component=0)

    return data_observ_points, observe_u


x_min, x_max = -2.0, 2.0
y_min, y_max = -0.89, 0.89

left_corner = np.array([x_min, y_min])  # xmin, ymin – Coordinate of bottom left corner.
right_corner = np.array([x_max, y_max])  # xmax, ymax – Coordinate of top right corner.
geometry_rectangle = dde.geometry.geometry_2d.Rectangle(left_corner, right_corner)

# Reduce the training domain to avoid pole singularities.
df_simulated_Holocene_2 = df_simulated_Holocene[
    (df_simulated_Holocene['lat'] >= -81) & (df_simulated_Holocene['lat'] <= 81)]



In [7]:

def pde(x, u):
    du_x = dde.grad.jacobian(u, x, j=0)  # du/dlambda
    du_y = dde.grad.jacobian(u, x, j=1)  # du/dtheta

    K = wind_latitude(x[:, 1:2])
    K = tf.cast(K, tf.float32)

    du_xx = dde.grad.hessian(u, x, i=0, j=0)  # d^2u/dlambda^2
    du_yy = dde.grad.hessian(u, x, i=1, j=1)  # d^2u/dtheta^2

    return (
        (-K * du_x * (1 / tf.cos(x[:, 1:2] * np.pi / 2)) +
         D * ((1 / (tf.cos(x[:, 1:2] * np.pi / 2) ** 2) * du_xx + du_yy - tf.tan(x[:, 1:2] * np.pi / 2) * du_y)))
    )


def space_boundary_north(x, on_boundary):
    return on_boundary and np.isclose(y_max, x[1])


def space_boundary_south(x, on_boundary):
    return on_boundary and np.isclose(y_min, x[1])


def periodic_boundary(x, domain):
    return domain and (np.isclose(x[0], x_min) or np.isclose(x[0], x_max))



In [8]:
def mse_metric(y_true, y_pred):
    if y_true is None or y_pred is None:
        return tf.constant(0.0)
    return tf.reduce_mean(tf.square(y_true - y_pred))


import os


def train_process(data_observ_points, observe_u, D, bc_1, bc_2, model_name):
    # Percorso cartella per salvataggio modelli e variabili
    save_dir = MODEL_SAVE_PATH + model_name + "/"

    # Crea la cartella se non esiste
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    data = dde.data.PDE(
        geometry_rectangle,
        pde,
        [observe_u, periodic_condition, periodic_condition_derivative, bc_1, bc_2],
        num_domain=2592,
        num_boundary=216,
        anchors=data_observ_points.train_x,
        train_distribution='uniform'
    )

    neurons = 32
    layer = 5
    layer_size = [2] + [neurons] * layer + [1]
    activation = "selu"
    initializer = "Glorot normal"
    net = dde.maps.FNN(layer_size, activation, initializer)
    model = dde.Model(data, net)
    dde.optimizers.set_LBFGS_options(maxcor=50, ftol=1e-20, maxiter=1e5)
    model.compile("adam", lr=0.00001, external_trainable_variables=[D, north_mean, south_mean],
                  loss_weights=[1, 10, 0.5, 0.5, 1, 1])

    # Train and save the model

    checkpointer = dde.callbacks.ModelCheckpoint(
        f"{MODEL_SAVE_PATH}{model_name}/{model_name}.ckpt",
        verbose=0, period=10000,
    )

    variable = dde.callbacks.VariableValue([D, north_mean, south_mean], period=10000,
                                           filename=MODEL_SAVE_PATH + model_name + "/variables.dat")

    losshistory, train_state = model.train(epochs=1000, callbacks=[variable, checkpointer])
    dde.saveplot(losshistory, train_state, issave=False, isplot=True)
    params = variable.get_value()

    return model, params, train_state.best_step


In [9]:
north_mean = dde.Variable(-1.0)
south_mean = dde.Variable(-2.0)
D = dde.Variable(1.0)

bc_1 = dde.DirichletBC(geometry_rectangle, lambda x: north_mean, space_boundary_north)
bc_2 = dde.DirichletBC(geometry_rectangle, lambda x: south_mean, space_boundary_south)
periodic_condition = dde.icbc.PeriodicBC(geom=geometry_rectangle, component_x=0, on_boundary=periodic_boundary,
                                         derivative_order=0)
periodic_condition_derivative = dde.icbc.PeriodicBC(geom=geometry_rectangle, component_x=0,
                                                    on_boundary=periodic_boundary, derivative_order=1)


In [10]:
from sklearn.model_selection import KFold

# Numero di bande
num_bands = 18




# Numero di strisce in cui dividere la regione
n_strisce = 18

# Estremi latitudinali della regione
lat_min, lat_max = -90,90
lon_min, lon_max = -180 ,180
# Calcolo gli estremi delle strisce (n_strisce+1 punti di divisione)
lon_bands = np.linspace(lon_min, lon_max, n_strisce + 1)

print("Limiti delle strisce longitudinali:", lon_bands)


bands = np.arange(num_bands)

# Imposta la cross-validation a 5 fold
kf = KFold(n_splits=5, shuffle=True, random_state=42)

fold = 1
mse_scores = []

for train_index, test_index in kf.split(bands):
    print(f"\n--- Fold {fold} ---")

    train_bands = bands[train_index]
    test_bands = bands[test_index]

    # Seleziona i punti corrispondenti alle bande
    df_train = df_simulated_Holocene_2[df_simulated_Holocene_2['lon_band'].isin(train_bands)]
    df_test = df_simulated_Holocene_2[df_simulated_Holocene_2['lon_band'].isin(test_bands)]

    print(f"Train bands: {train_bands}")
    print(f"Test bands: {test_bands}")
    print(f"Punti train: {len(df_train)}, test: {len(df_test)}")

    # Training
    data_obs, bc_point = training_points(df_train)
    pinn_model, _, _, _ = train_process(data_obs, bc_point, D, bc_1, bc_2, f'model_fold_{fold}')

    # Valutazione
    Xg = (df_test[['lon', 'lat']].values / 90.0).astype(np.float32)
    Yg = df_test['log_dep_norm'].values.reshape(-1, 1).astype(np.float32)
    pinn_g = pinn_model.predict(Xg)
    mse = np.mean((pinn_g - Yg) ** 2)
    mse_scores.append(mse)

    print(f"MSE Fold {fold}: {mse:.4f}")
    fold += 1


Limiti delle strisce longitudinali: [-180. -160. -140. -120. -100.  -80.  -60.  -40.  -20.    0.   20.   40.
   60.   80.  100.  120.  140.  160.  180.]

--- Fold 1 ---


KeyError: 'lon_band'